In [20]:
pip install striprtf


Note: you may need to restart the kernel to use updated packages.


In [21]:
from striprtf.striprtf import rtf_to_text
import json

# Path to the RTF file
rtf_file_path = r"C:\Users\Elakkiya\Downloads\algoparams_from_ui.json.rtf"
json_file_path = r"C:\Users\Elakkiya\Downloads\algoparams_from_ui.json"

# Read and extract the text from the RTF file
with open(rtf_file_path, "r") as rtf_file:
    rtf_content = rtf_file.read()

# Convert RTF content to plain text 
plain_text = rtf_to_text(rtf_content)

try:
    json_data = json.loads(plain_text)
    with open(json_file_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)
    print("RTF content has been successfully extracted and saved as JSON.")
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)


RTF content has been successfully extracted and saved as JSON.


In [6]:
import json

# Load the JSON file
json_file_path = r"C:\Users\Elakkiya\Downloads\algoparams_from_ui.json"
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# Extract the relevant target information from the nested structure
target_info = json_data.get("design_state_data", {}).get("target", {})
prediction_type = target_info.get("prediction_type", "Not Provided")
target_column = target_info.get("target", "Not Provided")
regression_type = target_info.get("type", "Not Provided")
partitioning = target_info.get("partitioning", False)

# Print the parsed values
print(f"Prediction Type: {prediction_type}")
print(f"Target Column: {target_column}")
print(f"Regression Type: {regression_type}")
print(f"Partitioning Enabled: {partitioning}")


Prediction Type: Regression
Target Column: petal_width
Regression Type: regression
Partitioning Enabled: True


In [7]:
import pandas as pd
import json

# Load the JSON file for feature handling
json_file_path = r"C:\Users\Elakkiya\Downloads\algoparams_from_ui.json"
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# Load the dataset 
csv_file_path = r"C:\Users\Elakkiya\Downloads\iris_modified.csv"
df = pd.read_csv(csv_file_path)

# Extract feature handling information from JSON
feature_handling = json_data.get("feature_handling", {})

# Iterate through the features in the JSON and handle missing values as required
for feature, feature_info in feature_handling.items():
    feature_name = feature_info.get("feature_name")
    missing_values = feature_info.get("feature_details", {}).get("missing_values", "")
    impute_with = feature_info.get("feature_details", {}).get("impute_with", "")
    impute_value = feature_info.get("feature_details", {}).get("impute_value", 0)

    # Check if the feature exists in the DataFrame
    if feature_name in df.columns:
        # Handle missing values
        if missing_values == "Impute":
            if impute_with == "Average of values":
                # Impute missing values with the mean of the column
                mean_value = df[feature_name].mean()
                df[feature_name].fillna(mean_value, inplace=True)
                print(f"Imputed missing values in '{feature_name}' with average value: {mean_value}")
            else:
                # Use provided impute_value if no mean-based imputation is defined
                df[feature_name].fillna(impute_value, inplace=True)
                print(f"Imputed missing values in '{feature_name}' with value: {impute_value}")
        else:
            print(f"No imputation required for '{feature_name}'")
    else:
        print(f"Feature '{feature_name}' not found in the dataset")

print("\nDataFrame after imputation:")
print(df)



DataFrame after imputation:
     sepal_length  sepal_width  petal_length  petal_width         species
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

[150 rows x 5 columns]


In [37]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None) 
print("\nDataFrame after imputation:")
print(df)



DataFrame after imputation:
     sepal_length  sepal_width  petal_length  petal_width          species
0             5.1          3.5           1.4          0.2      Iris-setosa
1             4.9          3.0           1.4          0.2      Iris-setosa
2             4.7          3.2           1.3          0.2      Iris-setosa
3             4.6          3.1           1.5          0.2      Iris-setosa
4             5.0          3.6           1.4          0.2      Iris-setosa
5             5.4          3.9           1.7          0.4      Iris-setosa
6             4.6          3.4           1.4          0.3      Iris-setosa
7             5.0          3.4           1.5          0.2      Iris-setosa
8             4.4          2.9           1.4          0.2      Iris-setosa
9             4.9          3.1           1.5          0.1      Iris-setosa
10            5.4          3.7           1.5          0.2      Iris-setosa
11            4.8          3.4           1.6          0.2      Iris-set

In [10]:
import pandas as pd
import json
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np

# Extract target and features from the JSON
target_column = json_data["design_state_data"]["target"]["target"]
X = df.drop(columns=[target_column])
y = df[target_column]

# Extract feature reduction information from JSON
feature_reduction = json_data.get("design_state_data", {}).get("feature_reduction", {})

# Determine the selected feature reduction method
selected_method = None
for method in feature_reduction:
    if method in ["No reduction", "Correlation with target", "Tree-based", "Principal Component Analysis"]:
        if feature_reduction[method].get("is_selected", False):
            selected_method = method
            break

print("Feature reduction configuration:", feature_reduction)


def apply_feature_reduction(method, X, y, config):
    if method == "No reduction":
        print("No reduction selected. Keeping all features.")
        return X  # No change

    elif method == "Correlation with target":
        num_features = config.get(method, {}).get("num_of_features_to_keep", X.shape[1])
        print(f"Applying Correlation with Target. Keeping top {num_features} features.")
        selector = SelectKBest(score_func=f_regression, k=num_features)
        X_new = selector.fit_transform(X, y)
        selected_columns = X.columns[selector.get_support()]
        return pd.DataFrame(X_new, columns=selected_columns)

    elif method == "Tree-based":
        num_features = int(config.get(method, {}).get("num_of_features_to_keep", X.shape[1]))
        num_trees = int(config.get(method, {}).get("num_of_trees", 100))
        depth_of_trees = int(config.get(method, {}).get("depth_of_trees", None))
        print(f"Applying Tree-based feature reduction. Keeping top {num_features} features.")
        model = RandomForestRegressor(n_estimators=num_trees, max_depth=depth_of_trees, random_state=42)
        model.fit(X, y)
        importances = model.feature_importances_
        indices = np.argsort(importances)[-num_features:]
        selected_columns = X.columns[indices]
        return X[selected_columns]

    elif method == "Principal Component Analysis":
        num_features = int(config.get(method, {}).get("num_of_features_to_keep", X.shape[1]))
        print(f"Applying PCA. Reducing to {num_features} components.")
        pca = PCA(n_components=num_features)
        X_new = pca.fit_transform(X)
        return pd.DataFrame(X_new, columns=[f'PC{i+1}' for i in range(num_features)])

    else:
        print(f"Unknown feature reduction method: {method}. No reduction applied.")
        return X  # No change


print(f"Selected feature reduction method: {selected_method}")

# Apply the selected feature reduction method
if selected_method:
    X_reduced = apply_feature_reduction(selected_method, X, y, feature_reduction)
else:
    print("No feature reduction method selected. Keeping all features.")
    X_reduced = X
    
print("\nDataFrame after feature reduction:")
print(X_reduced.head())


Feature reduction configuration: {'feature_reduction_method': 'Tree-based', 'num_of_features_to_keep': '4', 'num_of_trees': '5', 'depth_of_trees': '6'}
Selected feature reduction method: None
No feature reduction method selected. Keeping all features.

DataFrame after feature reduction:
   sepal_length  sepal_width  petal_length      species
0           5.1          3.5           1.4  Iris-setosa
1           4.9          3.0           1.4  Iris-setosa
2           4.7          3.2           1.3  Iris-setosa
3           4.6          3.1           1.5  Iris-setosa
4           5.0          3.6           1.4  Iris-setosa
